In [ ]:
!git clone https://github.com/Lou1sM/meaningful_image_complexity

Cloning into 'meaningful_image_complexity'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (314/314), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 314 (delta 190), reused 230 (delta 116), pack-reused 0 (from 0)
Receiving objects: 100% (314/314), 86.25 KiB | 5.07 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [ ]:
!git clone https://github.com/pytorch/vision.git

Cloning into 'vision'...
remote: Enumerating objects: 584408, done.
remote: Counting objects: 100% (32194/32194), done.
remote: Compressing objects: 100% (1892/1892), done.
remote: Total 584408 (delta 30158), reused 32042 (delta 30073), pack-reused 552214 (from 1)
Receiving objects: 100% (584408/584408), 1.09 GiB | 25.71 MiB/s, done.
Resolving deltas: 100% (546249/546249), done.


In [ ]:
!pip install pandas scipy matplotlib scikit-fuzzy scikit-image dl-utils385 --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 33.4 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="cniNWVazrZnzZa0rBAup")
project = rf.workspace("military-technical-academy-86a4o").project("uxo-app-detection")
version = project.version(55)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to UXO-App-Detection-55 in yolov8:: 100%|██████████| 6285/6285 [00:17<00:00, 350.76it/s] 


# Dataset Custom Loader

In [ ]:
import os
import torch
import cv2
import numpy as np
from torchvision.transforms import functional as F
from torchvision import transforms
from vision.references.detection.transforms import SimpleCopyPaste


def collate_fn(batch):
    return tuple(zip(*batch))

# Detection Input Size
input_size = (256, 256)

# CTX-UXO Dataset Polygon (Segmentents) to BBoxes
def polygon_to_bbox(polygon):
    x_coords = polygon[0::2]
    y_coords = polygon[1::2]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    return [x_min, y_min, x_max, y_max]

# Convert YOLO to TorchVision
def load_yolo_labels_with_polygons(label_path, image_shape):
    labels = []
    boxes = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            data = line.strip().split()
            class_id = int(data[0])
            polygon = list(map(float, data[1:]))
            bbox = polygon_to_bbox(polygon)


            # Rescalează bounding box-ul la dimensiunea specificată

            x_scale = input_size[0] / image_shape[1]
            y_scale = input_size[1] / image_shape[0]
            bbox = [bbox[0] * x_scale, bbox[1] * y_scale, bbox[2] * x_scale, bbox[3] * y_scale]

            boxes.append(bbox)
            labels.append(class_id)

    return boxes, labels

# Custom Dataset
class CTXUXODataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        label_path = os.path.join(self.labels_dir, os.path.splitext(self.image_files[idx])[0] + '.txt')

        # Load Image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        original_shape = image.shape[:2]

        # Convert YOLO Segments (Polygons) by calling predefined function
        boxes, labels = load_yolo_labels_with_polygons(label_path, original_shape)
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        if len(boxes) == 0:  # Dacă nu sunt obiecte, creează un tensor gol
            boxes = torch.empty((0, 4), dtype=torch.float32)  # 0 obiecte, 4 coordonate
            labels = torch.empty((0,), dtype=torch.int64)      # 0 obiecte


        # Transformă the image and labels
        if self.transform:
            image = self.transform(image)

        target = {
            'boxes': boxes,
            'labels': labels
        }
           #         'image_id': torch.tensor([idx]),
    #        'area': (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1]),
     #       'objects_count': len(labels)

        return image, target

# Copy Paste on new instance (mixup Albumentations)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(input_size),
    transforms.ToTensor()
])

# Dataset Config
def create_datasets(images_dir, labels_dir, train_split=0.8):
    image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg') or f.endswith('.png')]
    num_train = int(len(image_files) * train_split)

    train_dataset = CTXUXODataset(images_dir, labels_dir, transform=transform)
    val_dataset = CTXUXODataset(images_dir, labels_dir, transform=transform)

    train_dataset.image_files = image_files[:num_train]
    val_dataset.image_files = image_files[num_train:]

    return train_dataset, val_dataset

# Specificați calea către imaginile și etichetele dataset-ului
def get_dataset() ->list[str] :
   images_dir = '/content/UXO-App-Detection-55/train/images'
   labels_dir = '/content/UXO-App-Detection-55/train/labels'

   return images_dir, labels_dir

images_dir, labels_dir=get_dataset()

# Crearea dataset-urilor de antrenare și validare
train_dataset, val_dataset = create_datasets(images_dir, labels_dir)

# Exemplu de testare a dataset-ului
def test_dataset(dataset):
    for image, target in dataset:
        print(f"Image shape: {image.shape}, Target: {target}")
        break  # Doar primul exemplu

# Apel pentru testare
test_dataset(train_dataset)  # Pentru a testa dataset-ul de antrenare
test_dataset(val_dataset)     # Pentru a testa dataset-ul de validare

# Puteți folosi acum dataset-urile în DataLoader pentru antrenare
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=16,
                                                shuffle=True,
                                                collate_fn=collate_fn )
val_data_loader = torch.utils.data.DataLoader(val_dataset,
                                              batch_size=16,
                                              shuffle=False,
                                              collate_fn=collate_fn )


Image shape: torch.Size([3, 256, 256]), Target: {'boxes': tensor([[0.0345, 0.0649, 0.0479, 0.0762]]), 'labels': tensor([0])}
Image shape: torch.Size([3, 256, 256]), Target: {'boxes': tensor([[0.0656, 0.0607, 0.0951, 0.0687],
        [0.0261, 0.0523, 0.0417, 0.0703]]), 'labels': tensor([0, 0])}


# Image Complexity

In [ ]:
from meaningful_image_complexity.measure_complexity import ComplexityMeasurer
import numpy as np

def complexity_gate(image_path):
    comp_meas = ComplexityMeasurer(ncs_to_check=8,
                                  n_cluster_inits=1,
                                  nz=2,
                                  num_levels=4,
                                  cluster_model='GMM',
                                  info_subsample=0.3,
                                  )

    img = np.load(image_path)

    complexity_of_img_at_each_level = comp_meas.interpret(img)
    return complexity_of_img_at_each_level

#Moes

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, fasterrcnn_mobilenet_v3_large_fpn, ssdlite320_mobilenet_v3_large
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.optim as optim
import cv2
import numpy as np
from vision.references.detection import utils
from torch.utils.tensorboard import SummaryWriter
from meaningful_image_complexity.measure_complexity import ComplexityMeasurer


# Setări generale
input_size = (256, 256)
num_classes = 2  # UXO and Background


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(input_size),
    transforms.ToTensor()
])

# Gating Network pentru imagini complexe (region proposal)
class GatingNetworkComplex(nn.Module):
    def __init__(self, num_experts=10, input_dim=input_size[0]):
        super(GatingNetworkComplex, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        return torch.softmax(self.fc(x), dim=1)

# Gating Network pentru imagini simple (SSD)
class GatingNetworkSimple(nn.Module):
    def __init__(self, num_experts=10, input_dim=input_size[0]):
        super(GatingNetworkSimple, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        return torch.softmax(self.fc(x), dim=1)

# Expert bazat pe SSD (MobileNetV3) pentru imagini simple

# Expert bazat pe Faster R-CNN (MobileNetV3) pentru imagini complexe
class ExpertComplexRegionProposal(nn.Module):
    def __init__(self, num_classes=1):
        super(ExpertComplexRegionProposal, self).__init__()
         # Inițializează modelul SSDLite
        self.model = fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    def forward(self, x, targets=None):
        # Dacă se oferă ținte, se antrenează modelul
        if targets is not None:
            loss_dict = self.model(x, targets)
            return loss_dict
        else:
            # Se fac predicții
            predictions = self.model(x)
            return predictions

class ExpertSimpleSSD(nn.Module):
    def __init__(self, num_classes=1):
        super(ExpertSimpleSSD, self).__init__()
         # Inițializează modelul SSDLite
        self.model = ssdlite320_mobilenet_v3_large(weights="DEFAULT")
        in_features = self.model.head.classification_head.conv[0].in_channels
        # Schimbă predicatorul de boxe pentru a se potrivi cu numărul de clase
        self.model.head.classification_head = SSDLitePredictor(in_features, num_classes)

    def forward(self, x, targets=None):
        # Dacă se oferă ținte, se antrenează modelul
        if targets is not None:
            loss_dict = self.model(x, targets)
            return loss_dict
        else:
            # Se fac predicții
            predictions = self.model(x)
            return predictions


# Modelul complet Adaptive Mixture of Experts
class AdaptiveMixtureOfExperts(nn.Module):
    def __init__(self, threshold=0.5, num_experts=10, top_k=3):
        super(AdaptiveMixtureOfExperts, self).__init__()
        self.threshold = threshold
        self.gating_complex = GatingNetworkComplex(num_experts=num_experts)
        self.gating_simple = GatingNetworkSimple(num_experts=num_experts)
        # Experții
        self.experts_complex = nn.ModuleList([ExpertComplexRegionProposal() for _ in range(num_experts // 2)])  # 5 experți pentru imagini complexe
        self.experts_simple = nn.ModuleList([ExpertSimpleSSD() for _ in range(num_experts // 2)])  # 5 experți pentru imagini simple

    def forward(self, x, complexity_score=0.6):
        # Alege rețeaua de gating și experții în funcție de complexitate
        complexity_score=0.6 #TODO
        if complexity_score >= self.threshold:
            gating_network = self.gating_complex
            experts = self.experts_complex
        else:
            gating_network = self.gating_simple
            experts = self.experts_simple

        # Extrage caracteristici pentru gating
        output = experts[0].model.backbone(x)
        print(type(output))
        print(output.keys())
        features = output['1'].mean(dim=[2, 3])  # Caracteristici din backbone
        gating_weights = gating_network(features)

        # Selectează top K experți
        top_k_scores, top_k_indices = torch.topk(gating_weights, 3, dim=1)

        # Combina rezultatele experților
        outputs = torch.stack([expert(x) for expert in experts], dim=1)
        weighted_output = (outputs * gating_weights.unsqueeze(2)).sum(dim=1)

        return weighted_output




def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq, scaler=None):
    model.train()
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = f"Epoch: [{epoch}]"

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1.0 / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = torch.optim.lr_scheduler.LinearLR(
            optimizer, start_factor=warmup_factor, total_iters=warmup_iters
        )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in t.items()} for t in targets]
        with torch.cuda.amp.autocast(enabled=scaler is not None):
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print(f"Loss is {loss_value}, stopping training")
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        if scaler is not None:
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            losses.backward()
            optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger

# Exemplu de apel al funcției
# train_model(train_loader)  # Specificați train_loader
# test_single_image("cale/catre/imagine.jpg")


In [ ]:
import time
import math

def train_complex_expert():
    print("Start training for complex expert")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # model = AdaptiveMixtureOfExperts(threshold=0.5, num_experts=10).to(device)
    model=ExpertComplexRegionProposal().to(device)
    epochs=5
    criterion = nn.CrossEntropyLoss()
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.SGD(params,lr=0.0005,momentum=0.9,weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(
        optimizer,
        step_size=50,
        gamma=0.1
    )
    print_freq=20
    output_dir="./results"
    scaler = torch.cuda.amp.GradScaler()
    start_time = time.time()
    for epoch in range(epochs):
        train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq, scaler)
        lr_scheduler.step()
        if output_dir:
            checkpoint = {
                "model": model_without_ddp.state_dict(),
                "optimizer": optimizer.state_dict(),
                "lr_scheduler": lr_scheduler.state_dict(),
                "epoch": epoch,
                "epochs":epochs
            }
            utils.save_on_master(checkpoint, os.path.join(output_dir, f"model_{epoch}.pth"))
            utils.save_on_master(checkpoint, os.path.join(output_dir, "checkpoint.pth"))

        # evaluate after every epoch
        #evaluate(model, data_loader_test, device=device)

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print(f"Training time {total_time_str}")


In [ ]:
train_complex_expert()

Start training for complex expert
Epoch: [0]  [  0/157]  eta: 1:50:57  lr: 0.000004  loss: 0.4570 (0.4570)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.2237 (0.2237)  loss_rpn_box_reg: 0.2333 (0.2333)  time: 42.4063  data: 2.0506
Epoch: [0]  [ 20/157]  eta: 1:35:33  lr: 0.000068  loss: 0.4567 (0.4562)  loss_classifier: 0.0000 (0.0000)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.2225 (0.2223)  loss_rpn_box_reg: 0.2329 (0.2339)  time: 41.8195  data: 1.4360
